In [ ]:
#input must be options chain day1 - end

In [101]:
import os,h5py
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import joblib
import random
import hashlib

In [102]:
CSV_PATH     = './data/OptionsEOD.csv/'
PARQUET_PATH = './data/OptionsEOD.parquet'
PARQUET_STG_PATH = './data/OptionsEOD_STG.parquet'
SCALER_COL  = ['DTE','INTRINSIC_VALUE','TOTAL_VOLUME',	'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK', 'P_VOLUME' ]
ID = ['OPTIONS_ID']
SCALER_PATH = './data/scaler/'
UNIQUE_KEYS = ['QUOTE_DATE','SYMBOL','EXPIRE_DATE']
H5_PATH = './data/OptTrainData/'

In [103]:
#example
EOD_CSV = pd.read_csv(CSV_PATH+"qqq/qqq_eod_201201.txt", engine='pyarrow')
EOD_CSV.head()

,[QUOTE_UNIXTIME],[QUOTE_READTIME],[QUOTE_DATE],[QUOTE_TIME_HOURS],[UNDERLYING_LAST],[EXPIRE_DATE],[EXPIRE_UNIX],[DTE],[C_DELTA],[C_GAMMA],...,[P_LAST],[P_DELTA],[P_GAMMA],[P_VEGA],[P_THETA],[P_RHO],[P_IV],[P_VOLUME],[STRIKE_DISTANCE],[STRIKE_DISTANCE_PCT]
0,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.000000,-0.003130,0.002320,0.000350,-0.004080,0.000000,0.793590,,10.9,0.192
1,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.000000,-0.003580,0.002780,0.000530,-0.004420,-0.000420,0.721270,,9.9,0.174
2,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.000000,-0.004040,0.003340,0.000950,-0.004350,-0.000430,0.648640,,8.9,0.156
3,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,1.000000,0.000000,...,0.010000,-0.004390,0.004150,0.001170,-0.004050,-0.000330,0.577770,0.000000,7.9,0.139
4,1325624400,2012-01-03 16:00,2012-01-03,16.0,56.9,2012-01-06,1325883600,3.0,0.971540,0.016360,...,0.010000,-0.004860,0.005270,0.001370,-0.004140,0.000000,0.507560,0.000000,6.9,0.121


In [ ]:
EOD_CSV.columns

In [ ]:
#Part I
#TransformData : 
#-each partition from EXPIRE_DATE 
#-csv too parquet
#-col. rename 
def TransformDataI():
    scaler = MinMaxScaler()
    schema = None
    pqwriter = None
    for d in os.listdir(CSV_PATH):
        for f in os.listdir(CSV_PATH+f"{d}/"):
            if f.endswith(".txt"):
                ## load
                print( f"[LOAD] : {CSV_PATH}{d}/{f}        ",end='\r')
                EOD_CSV = pd.read_csv(CSV_PATH+f"{d}/"+f, engine='pyarrow')
                    
                ## rename col.
                for c in EOD_CSV.columns:
                    EOD_CSV = EOD_CSV.rename( columns={ c:c.strip().replace(']','').replace('[','') } )
                
                ## add symbol 
                EOD_CSV['SYMBOL'] = d.upper()
                ## add INTRINSIC_VALUE
                EOD_CSV['INTRINSIC_VALUE'] = EOD_CSV['UNDERLYING_LAST'] - EOD_CSV['STRIKE']
                
                ## fillnafillna
                EOD_CSV['P_VOLUME'] = EOD_CSV['P_VOLUME'].fillna(0)
                EOD_CSV['C_VOLUME'] = EOD_CSV['C_VOLUME'].fillna(0)


                
                # date columns convert to datetime
                for c in ["QUOTE_READTIME","QUOTE_DATE","EXPIRE_DATE"]:
                    EOD_CSV[c] = pd.to_datetime(EOD_CSV[c])
                
                #clean float data
                for c in ['INTRINSIC_VALUE','C_DELTA','C_GAMMA','C_VEGA','C_THETA','C_RHO','C_IV','C_VOLUME','C_LAST','C_BID','C_ASK','STRIKE','P_BID','P_ASK','P_LAST','P_DELTA','P_GAMMA','P_VEGA','P_THETA','P_RHO','P_IV','P_VOLUME','STRIKE_DISTANCE','STRIKE_DISTANCE_PCT']:
                    if EOD_CSV[c].dtype not in ( 'float32','float64'):
                        EOD_CSV[c] = EOD_CSV[c].apply(lambda x: x.strip())
                        EOD_CSV[c] = EOD_CSV[c].replace('', np.nan).fillna(np.nan)
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                    if EOD_CSV[c].dtype == 'float32':
                        EOD_CSV[c] = EOD_CSV[c].astype('float64')
                        
                # REMAIN_DAYS(int) =>  use DTE col.
                #partition with QUOTE_DATE
                EOD_CSV['PartitionDate'] = EOD_CSV['QUOTE_DATE'].dt.strftime('%Y-%m')
                EOD_CSV.sort_values(['QUOTE_DATE','EXPIRE_DATE','SYMBOL','STRIKE'],ascending =False ) 

                #scaler(Normalization_
                #scaler.partial_fit(EOD_CSV[SCALER_COL])

                # save
                if os.path.exists(PARQUET_PATH):
                  EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
                else:
                  EOD_CSV.to_parquet(PARQUET_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )
                    
    # joblib.dump(scaler, SCALER_PATH )
    # if pqwriter:
    #     pqwriter.close()
    # print( f"[DONE]                                                       ",end='\r')



In [ ]:
##-RunCleanData
#TransformDataI()

In [110]:
##### Part II 
#TransformData : 
# - read each partitions 
# - Normalization if not have scaler.gz file
def strikeZero(df,v,num_rm):
    # First, filter based on QUOTE_DATE, SYMBOL, and EXPIRE_DATE
    filtered_arr = df[(df['QUOTE_DATE'] == v['QUOTE_DATE']) &
                     (df['SYMBOL'] == v['SYMBOL']) &
                     (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])]['STRIKE'].values
    if len(filtered_arr) != 20:
        pass
    # print(max_intrinsic_value)
    # print(min_intrinsic_value)
    df.loc[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) 
            & (df['SYMBOL'] == v['SYMBOL']) 
            & (df['EXPIRE_DATE'] == v['EXPIRE_DATE'])
            & (  (df['STRIKE'].isin(filtered_arr[:5])  )
               | (df['STRIKE'].isin(filtered_arr[-5:])   )
              ) 
        , ['DTE', 'INTRINSIC_VALUE', 'C_BID',	'C_ASK', 'C_VOLUME',  'P_BID',	'P_ASK',	'P_VOLUME']       
    ] = 0
    
def hash_str(S):
    return hashlib.md5(S.encode('utf-8')).hexdigest() 
    

keys = None#df[unique_keys].sort_values(by=unique_keys).drop_duplicates()
max_option_len = 20
#==== Create MinMaxScaler
if not os.path.exists(SCALER_PATH):
    os.makedirs(SCALER_PATH)
Scaler = {}
for s in SCALER_COL:
    Scaler[s] = MinMaxScaler()
#==== Get PartitionDate
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
#PartitionDate = ['2011-12','2022-05'] # debug
options_qoute = {}
for i,partdate in enumerate(PartitionDate) :  
    df = pd.read_parquet(PARQUET_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', partdate)]
                                )
    #add col options_id
    df['OPTIONS_ID'] = None
    df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
    df['C_VOLUME'] = df['C_VOLUME'].fillna(0)
    df.dropna(subset=[c for c in SCALER_COL if c != 'TOTAL_VOLUME' ])
    ####################################################
    keys = df[UNIQUE_KEYS].sort_values(by=UNIQUE_KEYS).drop_duplicates()
    DATA  = np.empty((0,) + (20,9) ) #init 
    #loop each UNIQUE_KEYS keys 
    for j,v in keys.iterrows():
        df_filter=df[ (df['QUOTE_DATE'] == v['QUOTE_DATE']) & (df['SYMBOL'] == v['SYMBOL']) & (df['EXPIRE_DATE'] == v['EXPIRE_DATE']) ]
        df = df.drop(df_filter.index)
        qoute = "".join(v[ ['SYMBOL','EXPIRE_DATE'] ].apply(str).values)
        #add qoute
        if qoute not in [*options_qoute.keys()]:
            options_qoute[qoute] = {}
            options_qoute[qoute]['start_price'] = df_filter['UNDERLYING_LAST'].values[0]
            options_qoute[qoute]['strike'] = df_filter[ df_filter['INTRINSIC_VALUE'].abs().isin(df_filter['INTRINSIC_VALUE'].abs().sort_values()[:max_option_len]) ]['STRIKE'].values
            options_qoute[qoute]['exp'] = df_filter['EXPIRE_DATE'].values[0]
            #check diff UNDERLYING_LAST
            if df_filter['UNDERLYING_LAST'].values[0] != round(np.average(df_filter['UNDERLYING_LAST']),4):
                print('[ERROR] : set UNDERLYING_LAST ',qoute )
        #rm index max : max_option_len
        rm_strike_index = df_filter[ ~df_filter['STRIKE'].isin(options_qoute[qoute]['strike']) ].index
        df_filter = df_filter.drop(rm_strike_index)

        # Generate zero strike 
        # Generate a random float between 0.01 and 1
        random_number = random.uniform(0, 1)
        nom_rm_rows = [r  for r in [0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4] if random_number < r]
        strikeZero(df_filter,v,len(nom_rm_rows) )
        hash_id = hash_str(f'{i}{j}')
        #====filed rows
        if len(df_filter) > 15 :
            while len(df_filter) != 20:
                Even= -1
                if (len(df_filter) % 2) == 0:
                    Even = 0
                if len(df_filter) < 20 :
                    df_filter.loc[len(df_filter) ] = [0]*len(df_filter.columns)
                elif len(df_filter) > 20 :
                     df_filter = df_filter.drop(df_filter.index[Even])
        #add cal columns.      
        df_filter.loc[:, 'TOTAL_VOLUME'] = df_filter['P_VOLUME'].sum() + df_filter['C_VOLUME'].sum()
        df_filter.loc[:, 'DTE'] = df_filter[ df_filter['DTE'] != 0]['DTE'].mean()
        #df_filter.loc[:, 'OPTIONS_ID'] = hash_id
        
        if len(df_filter) == 20 and not np.isnan( np.sum( df_filter[SCALER_COL].values ) ) :
            DATA = np.vstack((DATA ,[df_filter[SCALER_COL]]))

    #========================== partial_fit ================================#
    [ [Scaler[s].partial_fit(DATA[:,:,i]),joblib.dump(scaler, SCALER_PATH+s+".gz" )] for i,s in enumerate(SCALER_COL) ]
    #=== clear expire options_qoute
    for qi in list(options_qoute.keys()):
        if options_qoute[qi]['exp'] < df_filter['QUOTE_DATE'].values[0]:
            options_qoute.pop(qi)

    #========================== SAVE DATA ================================#
    
    #=== save H5
    if not os.path.exists(H5_PATH):
        os.makedirs(H5_PATH)
    with h5py.File(H5_PATH+f"{partdate}.h5", 'w') as f:
        dset = f.create_dataset(f'{partdate}', data=DATA, chunks=True , compression='gzip')
    # #=== save parquet
    # if os.path.exists(PARQUET_STG_PATH):
    #   df.to_parquet(PARQUET_STG_PATH, engine='fastparquet', append=True, partition_cols=['PartitionDate'], index=False )
    # else:
    #   df.to_parquet(PARQUET_STG_PATH, engine='fastparquet' , partition_cols=['PartitionDate'], index=False  )   

    print(f"[Processing] {partdate}, {round(((i+1)/len(PartitionDate))*100,2)}%       ",end='\r')
    break

In [ ]:
++++++ TransformDataII DEBUG ++++++ 

In [ ]:
++++++ TransformDataII DEBUG ++++++ 

In [ ]:
#PartIII to H5

In [28]:
#Example
import random
from IPython.display import clear_output,display, HTML
#load scaler
scaler = MinMaxScaler()
if not os.path.exists(H5_PATH):
    os.makedirs(H5_PATH)
PartitionDate = [ d[-7:] for d in  os.listdir(PARQUET_PATH) if 'PartitionDate' in d]
random.shuffle(PartitionDate)
scaler = joblib.load(SCALER_PATH)
pd.options.mode.copy_on_write = True

for i,partdate in enumerate(PartitionDate) :
    df = pd.read_parquet(PARQUET_STG_PATH,engine='pyarrow'
                                 , filters=[('PartitionDate', '=', partdate)]
                                )
    df['P_VOLUME'] = df['P_VOLUME'].fillna(0)
    df['C_VOLUME'] = df['C_VOLUME'].fillna(0)
    DATA  = np.empty((0,) + (20,9) ) 
    for opt_id in np.unique( df[["OPTIONS_ID"]].values):
        df_filter  = df[df["OPTIONS_ID"]==opt_id]

        if len(df_filter) > 15 :
            while len(df_filter) != 20:
                Even= -1
                if (len(df_filter) % 2) == 0:
                    Even = 0
                if len(df_filter) < 20 :
                    df_filter.loc[len(df_filter) ] = [0]*len(df_filter.columns)
                    total_vol = df_filter['TOTAL_VOLUME'].values.mean()
                    df_filter.loc[:, 'TOTAL_VOLUME'] = total_vol
                    df_filter.loc[:, 'OPTIONS_ID']= opt_id
                elif len(df_filter) > 20 :
                     df_filter = df_filter.drop(df_filter.index[Even])

        
        if len(df_filter) == 20 and not np.isnan( np.sum( df_filter[SCALER_COL].values ) ) :
            DATA = np.vstack((DATA ,[scaler.transform(df_filter[SCALER_COL])]))
                    
    # Save the NumPy array to an HDF5 file
    with h5py.File(H5_PATH+f"{partdate}.h5", 'w') as f:
        dset = f.create_dataset(f'{partdate}', data=DATA, chunks=True , compression='gzip')

    print(f"[Processing] {partdate}, {round(((i+1)/len(PartitionDate))*100,2)}%     ",end='\r')

In [ ]:
3============== sampling show 3===========================

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Assume X is your data with multiple columns
# Specify custom scaling ranges for each column
scaling_ranges = {
    'column1': (0, 1000),  # Example: column1 has values between 0 and 1000
    'column2': (1e-04, 0.1),  # Example: column2 has values between 1e-04 and 0.1
    'column3': (0, 1),  # Example: column3 has values between 0 and 1
    # Add more columns as needed
}

# Initialize MinMaxScaler with custom scaling ranges
scalers = {}
for column, (min_val, max_val) in scaling_ranges.items():
    scaler = MinMaxScaler(feature_range=(min_val, max_val))
    scaler.partial_fit(X[[column]])  # Partial fit on each column individually
    scalers[column] = scaler

# Transform each column using its respective scaler
X_scaled = np.empty_like(X)
for column, scaler in scalers.items():
    X_scaled[:, X.columns.get_loc(column)] = scaler.transform(X[[column]])

# Use X_scaled for training your model

In [ ]:
=========================================== TEST ==============================================